In [1]:
# Anpassung des Plain-Styles in der Bibliothek Pybtex

import re
import io
from pybtex.database import parse_string
from pybtex.database.input import bibtex
from pybtex.plugin import find_plugin
from pybtex.style.formatting import BaseStyle, toplevel
from pybtex.style.formatting.plain import Style as PlainStyle
from pybtex.style.template import (
    field, first_of, href, join, optional, optional_field, sentence, tag,
    together, words, node, FieldIsMissing
)
from pybtex import richtext
from pybtex.richtext import Text, Symbol


# Benutzerspezifischer Stil, der von PlainStyle erbt
class CustomPlainStyle(PlainStyle):
    
    def get_inproceedings_template(self, e):        
        # Eigene Formatierung für inproceedings: Vor den Seitenzahlen soll "pages" als Wort erscheinen, nach den Seitenzahlen ein Komma statt Punkt.
        template = toplevel [
            sentence [self.format_names('author')],
            self.format_title(e, 'title'),
            words [
                'In',
                sentence [
                    optional[ self.format_editor(e, as_sentence=False) ],
                    self.format_btitle(e, 'booktitle', as_sentence=False),
                    self.format_volume_and_series(e, as_sentence=False),
                    optional[ self.format_chapter_and_pages(e) ],
                    self.format_address_organization_publisher_date(e),
                ],
            ],
            sentence [ optional_field('note') ],
            self.format_web_refs(e),
        ]
        return template

    # Überschreibe die Methode, um nach der Jahreszahl einen Punkt zu erzeugen, kein Komma.
    def get_inbook_template(self, e):
        template = toplevel [
            self.format_author_or_editor(e),
            sentence [
                self.format_btitle(e, 'title', as_sentence=False),
                self.format_chapter_and_pages(e),
            ],
            self.format_volume_and_series(e),
            sentence [
                field('publisher'),
                optional_field('address'),
                optional [
                    words [field('edition'), 'edition']
                ],
                field('year'), 
            ],
            optional_field('note'),
            self.format_web_refs(e),
        ]
        return template

    # Überschreibe die Methode, um nach der Jahreszahl einen Punkt zu erzeugen, kein Komma.
    def get_booklet_template(self, e):
        template = toplevel [
            self.format_names('author'),
            self.format_title(e, 'title'),
            sentence [
                optional_field('howpublished'),
                optional_field('address'),
                field('year'),
            ],
            optional_field('note'),
            self.format_web_refs(e),
        ]
        return template

In [5]:
# Beispiel-BibTeX-Code
bibtex_code = """
@inproceedings{example_booklet,
  pages = {1--2},
  author = {Jane Doe und John Smith},
  year = 2022,
  title = {Juli},
  address = {San Francisco, CA},
  booktitle = {Vorgestellt auf der Jahreskonferenz für Technologie},
  editor = {Bob Johnson}
}
"""

# BibTeX-Code parsen
bib_data = parse_string(bibtex_code, 'bibtex')

# Den benutzerdefinierten Formatter finden
plain_style = CustomPlainStyle()
plain_backend = find_plugin('pybtex.backends', 'text')()

# Einträge formatieren
formatted_entries = plain_style.format_entries(bib_data.entries.values())

# Custom Backend zum Erfassen der Ausgabe in einem String
class StringBackend(plain_backend.__class__):
    def __init__(self):
        super().__init__()
        self.output = io.StringIO()

    def write_entry(self, label, text):
        self.output.write(text + "\n")

    def get_output(self):
        return self.output.getvalue()

# Custom Backend verwenden
string_backend = StringBackend()

try:
    for entry in formatted_entries:
        string_backend.write_entry(entry.key, entry.text.render(string_backend))
except FieldIsMissing as ex:
    print(ex)

# Formatierten String erhalten
formatted_string = string_backend.get_output()
print(formatted_string)

Jane Doe und John Smith. Juli. In Bob Johnson, editor, Vorgestellt auf der Jahreskonferenz für Technologie, pages 1–2, San Francisco, CA, 2022.

